# Testing the Pipeline Package locally
This notebook shows how to use `LocalPipelineRunner` to test the components of the Pipeline Package in a local virtual Python environment.

The Pipeline Package used here was created in the [30-CreatePipelinePackage.ipynb](30-CreatePipelinePackage.ipynb) notebook.

The `LocalPipelineRunner` object takes the Pipeline Package and extracts its components.  
Once the components are extracted, you can run them individually by calling `run_component` with component name and structured input data.  
The method builds a `venv` Python virtual environment for running the component and installs the required dependencies listed in the `requirements` file defined for the component (`runtime_requirements.txt` as defined [30-CreatePipelinePackage.ipynb](30-CreatePipelinePackage.ipynb) notebook).  
Once the virtual python environment is ready, the method executes and feeds the component with your test data.

## Creating test payloads

First, we will generate test payloads to validate the pipeline's functionality.  
We will load data from the `historical_data.csv` file and create a list of payloads, with each payload containing 60 records.

In [ ]:
import pandas
import numpy as np

data = pandas.read_csv("../data/historical_data.csv")

input_tags = ["temperature_A", "temperature_B", "temperature_C", "valve_position_A", "valve_position_B"]

data = data[input_tags]
data_dicts = data.to_dict(orient='records')

payloads = [data_dicts[i:i + 60] for i in range(0, len(data_dicts), 60)]
print(np.array(payloads).shape)

The length of the payload list shows that we translated the historical data into 1440 payloads with 60 records in each.

## Testing the package locally

### Testing a single component

We can test the workflow with `LocalPipelineRunner`. We set the path to the Pipeline Package created in the [previous notebook](30-CreatePipelinePackage.ipynb), and run the single component we created there with a payload.


In [ ]:
from pathlib import Path
from simaticai.testing.pipeline_runner import LocalPipelineRunner

# Adjust the version number as needed
soft_sensor_on_edge_package = Path('../packages/SoftSensorPipeline-edge_1.zip')

test_dir = Path("../test")

with LocalPipelineRunner(soft_sensor_on_edge_package, test_dir) as runner:
    pipeline_output = runner.run_component("valve_controller", [payloads[0]])

Now we can check the calculated output for the first payload.

In [ ]:
pipeline_output

### Testing the pipeline

Alternatively, we can test the entire pipeline instead of a single component.

In [ ]:
with LocalPipelineRunner(soft_sensor_on_edge_package, test_dir) as runner:
    pipeline_output = runner.run_pipeline([payloads[0]])

In [ ]:
pipeline_output

### Testing the pipeline on the entire payload

Finally, we can run the whole Pipeline Package on the entire list of payloads.  
In this case the `LocalPipelineRunner` iterates over the elemnts of the `payloads` list and collects the provided outputs into a list

In [ ]:
with LocalPipelineRunner(soft_sensor_on_edge_package, test_dir) as runner:
    pipeline_output = runner.run_pipeline(payloads[:10])  # Run the first 10 payloads, but you can choose any valid range of the payloads

In [ ]:
pipeline_output